In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# pip install -q transformers

In [ ]:
dataset = pd.read_csv("/kaggle/input/postagger/POS.csv")

In [ ]:
import datasets
datasets.__version__

In [ ]:
dataset.head()

In [ ]:
label_list = dataset['labels'].unique().tolist()
print(len(label_list))

In [ ]:
label2ids = {label:i for i,label in enumerate(label_list)}
ids2label = {values:keys for keys,values in label2ids.items()}

In [ ]:
print(label2ids)
print(ids2label)

In [ ]:
dataset.isna().sum()

In [ ]:
index = dataset[dataset["words"].isna()].index.tolist()
print(index)

In [ ]:
dataset = dataset.drop(index=index)

In [ ]:
print(len(dataset["sentence_id"].unique()))

In [ ]:
grouped = dataset.groupby("sentence_id")
print(len(grouped["words"]))

In [ ]:
df_grouped = grouped.agg({
    "words":lambda x:list(x),
    "labels":lambda x:list(x)
})

In [ ]:
df_grouped.columns

In [ ]:
train,test = train_test_split(
    df_grouped,
    test_size = 0.2,
    shuffle = True
)

In [ ]:
train.reset_index(drop=True,inplace=True)

In [ ]:
train.head()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")


In [ ]:
print(train['words'].dtype)


In [ ]:
print(type(train["words"]))

In [ ]:
tokenized_input = tokenizer(train.iloc[0]["words"],is_split_into_words=True)

In [ ]:
print(tokenized_input.keys())

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

In [ ]:
word_ids = tokenized_input.word_ids(batch_index=0) 
print(word_ids)

In [ ]:
print(tokens)

In [ ]:
from datasets import Dataset
dataset_train = Dataset.from_pandas(train)
dataset_test = Dataset.from_pandas(test)

In [ ]:
print(type(dataset_train['words']))              # datasets.arrow_dataset.Column
print(type(dataset_train['words'][0]))           # list
print(type(dataset_train['words'][0][0]))        # str
print(type(dataset_train['labels'][0]))          # list
print(type(dataset_train['labels'][0][0]))       # str or int depending on mapping


In [ ]:
tokenized_input = tokenizer(dataset_train["words"][:100],is_split_into_words=True)

In [ ]:
# print(dataset_train['words'][0])
# for i,data in enumerate(dataset_train['labels'][0]):
#     print(f"{i}:{data}")
#     print(label2ids[data])
    
 

In [ ]:
def tokenize_and_align_labels(examples):
    tokenize_inputs = tokenizer(examples['words'][:],truncation=True,max_length = 512,is_split_into_words=True)
    labels = []

    for i,label in enumerate(examples['labels'][:]):
        word_ids = tokenize_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2ids[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenize_inputs['labels'] = labels
    return tokenize_inputs
                
        
    

In [ ]:

tokenized_inputs = dataset_train.map(tokenize_and_align_labels,batched=True,batch_size=100)